In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("DataCoSupplyChainDataset.csv", encoding = 'cp1252')

In [3]:
df = df.rename(columns = {'Product Card Id': 'Product ID', 'order date (DateOrders)': 'order_date', 
                          'Order Item Quantity': 'Total Units Sold', 'Sales': 'Total Sales', 'Order Profit Per Order': 'Total Profit'})

In [4]:
df['order_date'] = pd.to_datetime(df['order_date'], format = '%m/%d/%Y %H:%M')
df['Date'] = df['order_date'].dt.to_period('M').dt.to_timestamp()

In [5]:
first_appear = (df.groupby('Product ID')['Date'].min().reset_index(name = 'first_date'))
max_date = df['Date'].max()

rows = []
for _, r in first_appear.iterrows():
    pid   = r['Product ID']
    start = r['first_date']
    # all month-starts from first appearance through max_date
    for dt in pd.date_range(start = start, end = max_date, freq = 'MS'):
        rows.append({'Product ID': pid, 'Date': dt})

grid = pd.DataFrame(rows)

In [6]:
prod_master = (df[['Product ID', 'Product Name', 'Department Id', 'Category Id', 'Category Name', 'Product Price']]
    .drop_duplicates(subset = 'Product ID')
)

In [7]:
agg = (df.groupby(['Date', 'Product ID']).agg({'Total Units Sold': 'sum', 'Total Sales': 'sum', 'Total Profit': 'sum'}).reset_index())

In [8]:
full = (pd.merge(grid, agg, on = ['Product ID', 'Date'], how = 'left').fillna({'Total Units Sold': 0, 'Total Sales': 0.0, 'Total Profit':0.0}))

In [9]:
inv = full.merge(prod_master, on = 'Product ID', how = 'left')

In [10]:
rng = np.random.default_rng(seed = 42)
inv['Competitor Pricing'] = (inv['Product Price'] * rng.uniform(0.9, 1.1, size = len(inv))).round(2)
inv['Cost to Company'] = (inv['Product Price'] * rng.uniform(0.6, 0.8, size = len(inv))).round(2)

In [11]:
inv['month'] = inv['Date'].dt.month
inv['quarter'] = inv['Date'].dt.quarter
inv['is_month_start'] = inv['Date'].dt.is_month_start
inv['is_month_end'] = inv['Date'].dt.is_month_end

In [12]:
inv = inv.sort_values(['Product ID','Date']).reset_index(drop = True)
inv['seasonal_naive'] = inv.groupby('Product ID')['Total Units Sold'].shift(12)
inv['rolling_ma3'] = inv.groupby('Product ID')['Total Units Sold']\
                             .transform(lambda x: x.shift(1)
                                             .rolling(window = 3, min_periods = 1)
                                             .mean())

In [13]:
inv['Demand Forecast'] = ((0.2 * inv['seasonal_naive'] + 0.8 * inv['rolling_ma3']).round().fillna(inv['rolling_ma3']).fillna(0).round().astype(int))
inv.drop(columns = ['seasonal_naive', 'rolling_ma3'], inplace = True)

In [14]:
init_frac, order_frac = 1.2, 0.8

def flow_calc(group):
    group = group.sort_values('Date').copy()
    inv_lvls, units_ord, tot_inv = [], [], []
    prev_tot = None

    for _, row in group.iterrows():
        sold = row['Total Units Sold']
        fcst = row['Demand Forecast']

        if prev_tot is None:
            # init based on actual sales (never zero if sold>0)
            inv_lvl = int(init_frac * sold)
            prev_tot = inv_lvl
        else:
            inv_lvl = prev_tot

        # base order from forecast
        base_uo = int(order_frac * fcst)
        # bump up order if base_uo + on-hand < sold, to avoid negative
        uo = base_uo if (prev_tot + base_uo) >= sold else (sold - prev_tot)

        ti = prev_tot + uo - sold

        inv_lvls.append(inv_lvl)
        units_ord.append(uo)
        tot_inv.append(ti)

        prev_tot = ti

    group['Inventory Level'] = inv_lvls
    group['Units Ordered']    = units_ord
    group['Total Inventory']  = tot_inv
    return group

inv = (
    inv
    .groupby('Product ID', group_keys=False)
    .apply(flow_calc)
    .reset_index(drop=True)
)

C:\Users\neel9\AppData\Local\Temp\ipykernel_2612\2008329361.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inv


In [15]:
# 13) Reorder columns and export
final_cols = [
    'Date', 'Product ID', 'Product Name', 'Department Id', 'Category Id',
    'Category Name', 'Inventory Level', 'Total Units Sold', 'Total Sales',
    'Total Profit', 'Units Ordered', 'Demand Forecast', 'Product Price',
    'Competitor Pricing', 'Cost to Company', 'Total Inventory',
    'month', 'quarter', 'is_month_start', 'is_month_end'
]
inventory_df = inv[final_cols]

In [16]:
inventory_df

,Date,Product ID,Product Name,Department Id,Category Id,Category Name,Inventory Level,Total Units Sold,Total Sales,Total Profit,Units Ordered,Demand Forecast,Product Price,Competitor Pricing,Cost to Company,Total Inventory,month,quarter,is_month_start,is_month_end
0,2017-04-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,2.0,2.0,249.979996,106.320000,0.0,0,124.989998,131.84,91.43,0.0,4,2,True,False
1,2017-05-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,0.0,13.0,1624.869973,228.510005,13.0,2,124.989998,123.46,93.00,0.0,5,2,True,False
2,2017-06-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,0.0,13.0,1624.869973,458.999994,13.0,8,124.989998,133.95,91.03,0.0,6,2,True,False
3,2017-07-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,0.0,15.0,1874.849969,325.769995,15.0,9,124.989998,129.92,94.75,0.0,7,3,True,False
4,2017-08-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,0.0,17.0,2124.829964,-194.960013,17.0,14,124.989998,114.85,90.61,0.0,8,3,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,2018-01-01,1361,Toys,7,74,Toys,50.0,276.0,3185.039989,443.520002,226.0,253,11.540000,11.71,8.03,0.0,1,1,True,False
2446,2017-12-01,1362,Fighting video games,9,75,Video Games,273.0,228.0,9063.000000,750.370007,0.0,0,39.750000,36.17,25.61,45.0,12,4,True,False
2447,2018-01-01,1362,Fighting video games,9,75,Video Games,45.0,610.0,24247.500000,1967.149990,565.0,228,39.750000,38.99,27.88,0.0,1,1,True,False
2448,2017-12-01,1363,Summer dresses,4,76,Women's Clothing,326.0,272.0,58703.041986,7927.739958,0.0,0,215.820007,219.40,167.86,54.0,12,4,True,False


In [17]:
import numpy as np

inventory_df['Date'] = pd.to_datetime(inventory_df['Date'])
inventory_df = inventory_df.sort_values(['Product ID', 'Date']).reset_index(drop=True)

mask = inventory_df['Units Ordered'] == 0
inventory_df.loc[mask, 'Units Ordered'] = inventory_df.loc[mask, 'Total Units Sold'] * np.random.uniform(0.9, 1.1, size=mask.sum())

inventory_df['Units Ordered'] = inventory_df['Units Ordered'].round(0)

mask = inventory_df['Demand Forecast'] < inventory_df['Units Ordered']
inventory_df.loc[mask, 'Demand Forecast'] = inventory_df.loc[mask, 'Units Ordered'] * np.random.uniform(1.1, 1.5, size = mask.sum())

inventory_df['Demand Forecast'] = inventory_df['Demand Forecast'].round(0)

def update_inventory(group):
    group = group.sort_values('Date').copy()
    inv_lvls = []
    tot_inv = []
    prev_tot = 0
    for _, row in group.iterrows():
        inv_lvl = prev_tot + row['Demand Forecast']
        total_inventory = inv_lvl - row['Total Units Sold']
        inv_lvls.append(inv_lvl)
        tot_inv.append(total_inventory)
        prev_tot = total_inventory
    group['Inventory Level'] = [round(x, 0) for x in inv_lvls]
    group['Total Inventory'] = [round(x, 0) for x in tot_inv]
    return group

inventory_df = inventory_df.groupby('Product ID', group_keys=False).apply(update_inventory).reset_index(drop=True)
inventory_df

C:\Users\neel9\AppData\Local\Temp\ipykernel_2612\3499703856.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.65998745e+00 1.66112385e+01 1.90332643e+01 1.84146594e+01
 2.18496668e+01 1.63443449e+01 8.07144454e+01 7.71052078e+01
 6.51141552e+00 1.82444787e+01 1.63193730e+01 1.67615614e+01
 2.59786707e+01 3.85814341e+01 3.68904193e+01 3.75533876e+01
 4.72795862e+01 4.09569502e+01 5.47518198e+01 4.64240309e+01
 6.45185939e+01 3.59245396e+01 4.80025768e+01 6.54318938e+01
 5.98938587e+01 5.98260731e+01 4.09475307e+01 3.80290511e+01
 5.14425719e+01 5.69793700e+01 5.87983305e+01 4.84180281e+01
 4.56538628e+01 4.07417122e+01 3.65215220e+01 4.78715119e+01
 3.33153690e+01 2.44787564e+00 1.10799415e+01 3.38699867e+01
 1.52273683e+01 5.89342296e+01 5.09639282e+01 4.53971318e+01
 4.24911167e+01 6.11540526e+01 5.36249261e+01 4.74021362e+01
 3.24177860e+01 5.03869588e+01 4.34892169e+01 4.38586633e+01
 6.16492433

,Date,Product ID,Product Name,Department Id,Category Id,Category Name,Inventory Level,Total Units Sold,Total Sales,Total Profit,Units Ordered,Demand Forecast,Product Price,Competitor Pricing,Cost to Company,Total Inventory,month,quarter,is_month_start,is_month_end
0,2017-04-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,3.0,2.0,249.979996,106.320000,2.0,3.0,124.989998,131.84,91.43,1.0,4,2,True,False
1,2017-05-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,18.0,13.0,1624.869973,228.510005,13.0,17.0,124.989998,123.46,93.00,5.0,5,2,True,False
2,2017-06-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,24.0,13.0,1624.869973,458.999994,13.0,19.0,124.989998,133.95,91.03,11.0,6,2,True,False
3,2017-07-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,29.0,15.0,1874.849969,325.769995,15.0,18.0,124.989998,129.92,94.75,14.0,7,3,True,False
4,2017-08-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,36.0,17.0,2124.829964,-194.960013,17.0,22.0,124.989998,114.85,90.61,19.0,8,3,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,2018-01-01,1361,Toys,7,74,Toys,295.0,276.0,3185.039989,443.520002,226.0,253.0,11.540000,11.71,8.03,19.0,1,1,True,False
2446,2017-12-01,1362,Fighting video games,9,75,Video Games,250.0,228.0,9063.000000,750.370007,219.0,250.0,39.750000,36.17,25.61,22.0,12,4,True,False
2447,2018-01-01,1362,Fighting video games,9,75,Video Games,761.0,610.0,24247.500000,1967.149990,565.0,739.0,39.750000,38.99,27.88,151.0,1,1,True,False
2448,2017-12-01,1363,Summer dresses,4,76,Women's Clothing,333.0,272.0,58703.041986,7927.739958,280.0,333.0,215.820007,219.40,167.86,61.0,12,4,True,False


In [18]:
inventory_df = inventory_df[inventory_df['Date'] <= '2017-04-30']
inventory_df.rename(columns = {"Demand Forecast": "Inventory Forecast"}, inplace = True)
inventory_df.reset_index(drop = True)

C:\Users\neel9\AppData\Local\Temp\ipykernel_2612\280946684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory_df.rename(columns = {"Demand Forecast": "Inventory Forecast"}, inplace = True)


,Date,Product ID,Product Name,Department Id,Category Id,Category Name,Inventory Level,Total Units Sold,Total Sales,Total Profit,Units Ordered,Inventory Forecast,Product Price,Competitor Pricing,Cost to Company,Total Inventory,month,quarter,is_month_start,is_month_end
0,2017-04-01,19,Nike Men's Fingertrap Max Training Shoe,2,2,Soccer,3.0,2.0,249.979996,106.320000,2.0,3.0,124.989998,131.84,91.43,1.0,4,2,True,False
1,2017-04-01,24,Elevation Training Mask 2.0,2,2,Soccer,16.0,12.0,959.879982,-90.479999,11.0,16.0,79.989998,77.92,61.96,4.0,4,2,True,False
2,2017-04-01,35,adidas Brazuca 2014 Official Match Ball,2,3,Baseball & Softball,7.0,5.0,799.950028,122.730002,5.0,7.0,159.990005,168.25,110.27,2.0,4,2,True,False
3,2015-01-01,37,adidas Kids' F5 Messi FG Soccer Cleat,2,3,Baseball & Softball,26.0,18.0,629.820015,-305.269996,18.0,26.0,34.990002,36.70,24.53,8.0,1,1,True,False
4,2015-02-01,37,adidas Kids' F5 Messi FG Soccer Cleat,2,3,Baseball & Softball,26.0,20.0,699.800022,174.660004,17.0,18.0,34.990002,38.26,22.64,6.0,2,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,2016-12-01,1073,Pelican Sunstream 100 Kayak,7,48,Water Sports,2505.0,446.0,89195.542453,12422.839960,446.0,483.0,199.990005,207.74,156.35,2059.0,12,4,True,False
1541,2017-01-01,1073,Pelican Sunstream 100 Kayak,7,48,Water Sports,2526.0,432.0,86395.682376,9042.300007,432.0,467.0,199.990005,213.90,143.74,2094.0,1,1,True,False
1542,2017-02-01,1073,Pelican Sunstream 100 Kayak,7,48,Water Sports,2540.0,443.0,88595.572436,12089.199881,443.0,446.0,199.990005,189.49,131.97,2097.0,2,1,True,False
1543,2017-03-01,1073,Pelican Sunstream 100 Kayak,7,48,Water Sports,2731.0,495.0,98995.052722,10440.139997,495.0,634.0,199.990005,186.01,147.28,2236.0,3,1,True,False


In [19]:
inventory_df.to_csv('inventory_final.csv', index = False)